# Additional File No.1  

<b> <font color='purple'> 
    
  This file contains some Cost Sensitive Algorithms and some Data Sampling Algorithms. These algorithms are run here and then loaded into the oringinal file using joblib inorder take advantage of parallel running of algorithms as this saves time

# Cost Sensitive Algorithms

In [96]:
ftwo_scorer = make_scorer(fbeta_score, beta=2)

## XgBoost Trial 1

In [90]:
xgbc= XGBClassifier(random_state=0,early_stopping_rounds=2,objective= 'binary:logistic',class_weight='balanced')
xgbc_param = {
              'max_depth' : [6,7,8,9],
              'n_estimators' : [50,100,150],
              'learning_rate' : [0.4, 0.6,0.8],
               'min_child_weight' : [1,3],
                'subsample':[0.8,0.9,1]
             }
xgbc_grid_bal = GridSearchCV(xgbc, xgbc_param,cv=5, return_train_score=True,scoring=ftwo_scorer)
xgbc_grid_bal.fit(X_train,y_train)

# Mean Cross Validation Score
print("Best Mean Cross-validation score: {:.2f}".format(xgbc_grid_bal.best_score_))
print()

#find best parameters
print('XG Boost parameters: ',xgbc_grid_bal.best_params_)


# Check test data set performance
print("Train score of XG Boost: ",xgbc_grid_bal.score(X_train,y_train))
print("Test score of XG Boost: ",xgbc_grid_bal.score(X_val,y_val))

Best Mean Cross-validation score: 0.86

XG Boost parameters:  {'learning_rate': 0.6, 'max_depth': 8, 'min_child_weight': 1, 'n_estimators': 50, 'subsample': 1}
Train score of XG Boost:  1.0
Test score of XG Boost:  0.8213256484149856


In [91]:
## To Save XGBoost Cost sensitive
dump(xgbc_grid_bal, 'XGboost_costsensitive1.joblib') 


['XGboost_costsensitive1.joblib']

## XgBoost Trial 2

In [101]:
xgboost = XGBClassifier()
param_grid = {
    
    'scale_pos_weight': [1, 2, 5, 50, 75, 100],
}

#apply grid search
grid_xgboost_bal = GridSearchCV(xgboost, param_grid, cv=6, n_jobs=2, scoring=ftwo_scorer)
grid_xgboost_bal.fit(X_train,y_train)

print("Best parameters: {}".format(grid_xgboost_bal.best_params_))
print("Best Mean cross-validation score: {:.2f}".format(grid_xgboost_bal.best_score_))

# Check test data set performance
print("Train score of XG Boost: ",grid_xgboost_bal.score(X_train,y_train))
print("Test score of XG Boost: ",grid_xgboost_bal.score(X_val,y_val))

Best parameters: {'scale_pos_weight': 75}
Best Mean cross-validation score: 0.85
Train score of XG Boost:  1.0
Test score of XG Boost:  0.8189655172413792


# Data Sampling Algorithms - Oversampling

## Logistic Regression with oversampling

In [83]:
# GridSearch with oversampling
pipe_log_smote = Pipeline([('smote', SMOTE()), ('model',LogisticRegression())])

param_grid = {
    # try different feature engineering parameters
    'smote__k_neighbors': [1,2,3,4,5],
    'model__C':  [5, 8, 10, 20, 100, 150, 200],
    'model__penalty':  ['l1','l2'],
}

#apply grid search
grid_smote_log = GridSearchCV(pipe_log_smote, param_grid, cv=5, n_jobs=2, scoring=ftwo_scorer)
grid_smote_log.fit(X_train,y_train)

# Mean Cross validation Score
print("Best Mean cross-validation score: {:.2f}".format(grid_smote_log.best_score_))

#find best parameters
print("Best parameters: {}".format(grid_smote_log.best_params_))

# Check test data set performance
print(f'Train score is {grid_smote_log.score(X_train,y_train)}')
print(f'Test score is {grid_smote_log.score(X_val,y_val)}')

Best Mean cross-validation score: 0.65
Best parameters: {'model__C': 200, 'model__penalty': 'l2', 'smote__k_neighbors': 5}
Train score is 0.6663872590108969
Test score is 0.6412825651302605


In [106]:
## To Save XGBoost data sampling
dump(grid_smote_log, 'Log_Smote.joblib') 

['Log_Smote.joblib']

## Decision Tree with oversampling

### Decision Tree with smote

In [85]:
pipe_dec_svmsmote = Pipeline([('svmsmote', SVMSMOTE()), ('model', DecisionTreeClassifier(random_state=0))])

param_grid = {
    # try different feature engineering parameters
    'svmsmote__k_neighbors': [1,2,3,4,5],
    'model__max_depth': range(3,36),
}

#apply grid search
grid_smote_deci1 = GridSearchCV(pipe_dec_svmsmote, param_grid, cv=5, n_jobs=2, scoring=ftwo_scorer)
grid_smote_deci1.fit(X_train,y_train)


# Mean Cross validation Score
print("Best Mean cross-validation score: {:.2f}".format(grid_smote_deci1.best_score_))

#find best parameters
print("Best parameters: {}".format(grid_smote_deci1.best_params_))

# Check test data set performance
print(f'Train score is {grid_smote_deci1.score(X_train,y_train)}')
print(f'Test score is {grid_smote_deci1.score(X_val,y_val)}')

Best Mean cross-validation score: 0.81
Best parameters: {'model__max_depth': 34, 'svmsmote__k_neighbors': 2}
Train score is 1.0
Test score is 0.7596685082872927


In [107]:
## To Save dtree data sampling
dump(grid_smote_deci1, 'dtree_Smote.joblib') 

['dtree_Smote.joblib']

### Decision Tree with SVMsmote

In [97]:
pipe_dec_svmsmote = Pipeline([('svmsmote', SVMSMOTE()), ('model', DecisionTreeClassifier(random_state=0))])

param_grid = {
    # try different feature engineering parameters
    'svmsmote__k_neighbors': [1,2,3],
    'model__max_depth': range(3,36),
     'model__criterion': ['gini', 'entropy'],
     'model__splitter': ['best', 'random']
}

#apply grid search
grid_svmsmote_deci1 = GridSearchCV(pipe_dec_svmsmote, param_grid, cv=6, n_jobs=2, scoring=ftwo_scorer)
grid_svmsmote_deci1.fit(X_train,y_train)


# Mean Cross validation Score
print("Best Mean cross-validation score: {:.2f}".format(grid_svmsmote_deci1.best_score_))

#find best parameters
print("Best parameters: {}".format(grid_svmsmote_deci1.best_params_))

# Check test data set performance
print(f'Train score is {grid_svmsmote_deci1.score(X_train,y_train)}')
print(f'Test score is {grid_svmsmote_deci1.score(X_val,y_val)}')

Best Mean cross-validation score: 0.84
Best parameters: {'model__criterion': 'entropy', 'model__max_depth': 17, 'model__splitter': 'best', 'svmsmote__k_neighbors': 2}
Train score is 1.0
Test score is 0.782967032967033


In [100]:
## To Save dtree data sampling
from joblib import dump, load
dump(grid_svmsmote_deci1, 'dtree_SVMSmote.joblib') 

['dtree_SVMSmote.joblib']

### Decision Tree with ADASYN

In [87]:
pipe_deci_adasyn = Pipeline([('adasyn', ADASYN()), ('model', DecisionTreeClassifier(random_state=0))])


param_grid = {
    # try different feature engineering parameters
    'adasyn__n_neighbors': [1,2,3,4,5],
    'model__max_depth': range(3,36),
}

#apply grid search
grid_adasyn_deci = GridSearchCV(pipe_deci_adasyn, param_grid, cv=8, n_jobs=2, scoring=ftwo_scorer)
grid_adasyn_deci.fit(X_train,y_train)

# Mean Cross validation Score
print("Best Mean cross-validation score: {:.2f}".format(grid_adasyn_deci.best_score_))

#find best parameters
print("Best parameters: {}".format(grid_adasyn_deci.best_params_))

# Check test data set performance
print(f'Train score is {grid_adasyn_deci.score(X_train,y_train)}')
print(f'Test score is {grid_adasyn_deci.score(X_val,y_val)}')

Best Mean cross-validation score: 0.71
Best parameters: {'adasyn__n_neighbors': 2, 'model__max_depth': 34}
Train score is 0.9873708381171066
Test score is 0.7179487179487181


In [108]:
## To Save dtree data sampling
dump(grid_adasyn_deci, 'dtree_ADASmote.joblib') 

['dtree_ADASmote.joblib']

## KNN with oversampling

In [88]:
pipe_knn_smote = Pipeline([('smote', SMOTE()), ('model',KNeighborsClassifier())])

param_grid = {
    # try different feature engineering parameters
    'smote__k_neighbors': [1,2,3,4,5],
    'model__n_neighbors': range(2,45),
}

#apply grid search
grid_smote_knn = GridSearchCV(pipe_knn_smote, param_grid, cv=5, n_jobs=2, scoring=ftwo_scorer)
grid_smote_knn.fit(X_train,y_train)

# Mean Cross validation Score
print("Best Mean cross-validation score: {:.2f}".format(grid_smote_knn.best_score_))

#find best parameters
print("Best parameters: {}".format(grid_smote_knn.best_params_))

# Check test data set performance
print(f'Train score is {grid_smote_knn.score(X_train,y_train)}')
print(f'Test score is {grid_smote_knn.score(X_val,y_val)}')

Best Mean cross-validation score: 0.79
Best parameters: {'model__n_neighbors': 4, 'smote__k_neighbors': 3}
Train score is 0.9587513935340023
Test score is 0.7772020725388602


In [109]:
## To Save knn data sampling
dump(grid_smote_knn, 'knn_Smote.joblib') 

['knn_Smote.joblib']

## SVC 

### SVC Linear

pipe_SVCL_smote = Pipeline([('smote', SMOTE()), ('model', SVC(kernel = 'linear'))]) <br>

param_grid = {<br>
    # try different feature engineering parameters<br>
    'smote__k_neighbors': [1,2,3,4,5],<br>
    'model__C': [0.8, 1, 1.5, 10, 15],<br>
    'model__gamma':[0.0001,0.001,0.1],<br>
    
}<br>

#apply grid search<br>
grid_SVCL_smote = GridSearchCV(pipe_SVCL_smote, param_grid, cv=5, n_jobs=2, scoring=ftwo_scorer)<br>
grid_SVCL_smote.fit(X_train,y_train)<br>

Mean Cross validation Score<br>
print("Best Mean cross-validation score: {:.2f}".format(grid_SVCL_smote.best_score_))<br>

find best parameters<br>
print("Best parameters: {}".format(grid_SVCL_smote.best_params_))<br>

Check test data set performance<br>
print(f'Train score is {grid_SVCL_smote.score(X_train,y_train)}')<br>
print(f'Test score is {grid_SVCL_smote.score(X_val,y_val)}')<br>

<font color='purple'> <b> ** Since SVC code took more than 12 hrs to run, it is not run <br>

# Prediction on Test Data

In [97]:
X_test=test.drop(['Id'], axis=1)
X_test.shape

(24846, 29)

In [98]:
# Apply Transformations
X_test=data_preprocess.transform(X_test)

In [102]:
kaggle_model = XGBClassifier(random_state=0,early_stopping_rounds=2,objective= 'binary:logistic', 
                             learning_rate = 0.6, max_depth = 8, min_child_weight = 1, n_estimators = 50,
                             subsample = 1)
kaggle_model.fit(X_train,y_train)

# Preprocessing of validation data, get predictions
test_data_labels = kaggle_model.predict(X_test)

# Create predictions to be submitted!
pd.DataFrame({'Id': test.Id, 'Target': test_data_labels}).to_csv('solution_base_xgb_normal.csv', index =False)  
